In [1]:
import pandas as pd
from sqlalchemy.engine import URL # ao usar pyodbc o pandas continuava dando aviso recomendando usar sqlalchemy, portanto mudei a biblioteca utilizada para a query
from sqlalchemy import create_engine, text
import boto3
import ast

In [ ]:
secret_name = "database_tcepb"
region_name = "sa-east-1"

# Create a Secrets Manager client
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)
SERVER_TRIBUNAL = ast.literal_eval(get_secret_value_response["SecretString"])["host"] + ", " + ast.literal_eval(get_secret_value_response["SecretString"])["port"]
DB_TRIBUNAL = ast.literal_eval(get_secret_value_response["SecretString"])["dbname"]
USERNAME = ast.literal_eval(get_secret_value_response["SecretString"])["username"]
PASSWORD = ast.literal_eval(get_secret_value_response["SecretString"])["password"]

In [2]:
string_conexao = "DRIVER={SQL Server};" + f"SERVER={SERVER_TRIBUNAL};DATABASE={DB_TRIBUNAL};UID={USERNAME};PWD={PASSWORD}"
url_conexao = URL.create("mssql+pyodbc", query={"odbc_connect": string_conexao}) # cria uma URL de conexão para ser colocado em engine
engine = create_engine(url_conexao) # Documentation: https://docs.sqlalchemy.org/en/14/core/engines.html

In [3]:
cnpj = "'11308834000185'"

### Queries

In [4]:
query_notas_canceladas = \
rf"""
SELECT
	COUNT(DISTINCT chave.chave) AS qtde_notas_canceladas
FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
	ON nfe.id_chave = chave.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
	ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
	ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
WHERE
	nfe.nota_cancelada = 1 AND
	cpf_cnpj.cnpj = {cnpj} AND
    chave.chave IS NOT NULL
"""

In [5]:
query_notas_ativas = \
rf"""
SELECT
	COUNT(DISTINCT chave.chave) AS qtde_notas_ativas
FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
	ON nfe.id_chave = chave.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
	ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
	ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
WHERE
	nfe.nota_cancelada = 0 AND
	nfe.id_finalidade = 1 AND
	cpf_cnpj.cnpj = {cnpj} AND
	chave.chave NOT IN (
		SELECT
			DISTINCT chave.chave
		FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
			ON nfe.id_chave = chave.id_chave
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
			ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
			ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
		WHERE
			nfe.nota_cancelada = 1 AND
			cpf_cnpj.cnpj = {cnpj}
	) AND
    chave.chave IS NOT NULL
"""

In [6]:
query_itens_ativos = \
rf"""
SELECT
	COUNT(PRODUTOS_ATIVOS.id_produto) AS qtde_itens_ativos
FROM(
	SELECT
		produtos.id_chave,
		produtos.id_produto,
		produtos.quantidade
	FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_PRODUTO AS produtos -- Carregando a tabela produtos que é o alvo da query

	INNER JOIN (
		SELECT
			MIN(chave.id_chave) AS id_chave,
			chave.chave
		FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
			ON nfe.id_chave = chave.id_chave
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
			ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
			ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
		WHERE
			nfe.nota_cancelada = 0 AND
			nfe.id_finalidade = 1 AND
			cpf_cnpj.cnpj = {cnpj} AND
			chave.chave NOT IN (
				SELECT
					DISTINCT chave.chave
				FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
				INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
					ON nfe.id_chave = chave.id_chave
				INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
					ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
				INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
					ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
				WHERE
					nfe.nota_cancelada = 1 AND
					cpf_cnpj.cnpj = {cnpj}
			)
		GROUP BY chave.chave
	) AS ATIVAS_NORMAIS
		ON produtos.id_chave = ATIVAS_NORMAIS.id_chave
) AS PRODUTOS_ATIVOS
WHERE PRODUTOS_ATIVOS.id_produto IS NOT NULL
"""

In [7]:
query_produtos_ativos = \
rf"""
SELECT
	SUM(produtos.quantidade) AS qtde_produtos_ativos
FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_PRODUTO AS produtos -- Carregando a tabela produtos que é o alvo da query
INNER JOIN (
	SELECT
		MIN(chave.id_chave) AS id_chave,
		chave.chave
	FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
	INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
		ON nfe.id_chave = chave.id_chave
	INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
		ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
	INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
		ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
	WHERE
		nfe.nota_cancelada = 0 AND
		nfe.id_finalidade = 1 AND
		cpf_cnpj.cnpj = {cnpj} AND
		chave.chave NOT IN (
			SELECT
				DISTINCT chave.chave
			FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
			INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
				ON nfe.id_chave = chave.id_chave
			INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
				ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
			INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
				ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
			WHERE
				nfe.nota_cancelada = 1 AND
				cpf_cnpj.cnpj = {cnpj}
		)
	GROUP BY chave.chave
) AS ATIVAS_NORMAIS
	ON produtos.id_chave = ATIVAS_NORMAIS.id_chave
WHERE produtos.quantidade IS NOT NULL
"""

In [8]:
query_notas_devolvidas = \
rf"""
SELECT
	COUNT(DISTINCT chave.chave) AS qtde_notas_devolvidas
FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
	ON nfe.id_chave = chave.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
	ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
	ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
WHERE
	nfe.nota_cancelada = 0 AND
	nfe.id_finalidade = 4 AND
	cpf_cnpj.cnpj = {cnpj} AND
	chave.chave NOT IN (
		SELECT
			DISTINCT chave.chave
		FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
			ON nfe.id_chave = chave.id_chave
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
			ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
			ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
		WHERE
			nfe.nota_cancelada = 1 AND
			cpf_cnpj.cnpj = {cnpj}
		)
    AND chave.chave IS NOT NULL
"""

In [9]:
query_itens_devolvidos = \
rf"""
SELECT
	COUNT(PRODUTOS_ATIVOS_DEVOLVIDOS.id_produto) AS qtde_itens_devolvidos
FROM(
	SELECT
		produtos.id_chave,
		produtos.id_produto,
		produtos.quantidade
	FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_PRODUTO AS produtos -- Carregando a tabela produtos que é o alvo da query
	INNER JOIN (
		SELECT
			MIN(chave.id_chave) AS id_chave,
			chave.chave
		FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
			ON nfe.id_chave = chave.id_chave
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
			ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
			ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
		WHERE
			nfe.nota_cancelada = 0 AND
			nfe.id_finalidade = 4 AND
			cpf_cnpj.cnpj = {cnpj} AND
			chave.chave NOT IN (
				SELECT
					DISTINCT chave.chave
				FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
				INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
					ON nfe.id_chave = chave.id_chave
				INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
					ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
				INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
					ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
				WHERE
					nfe.nota_cancelada = 1 AND
					cpf_cnpj.cnpj = {cnpj}
				)
			GROUP BY chave.chave
	) AS ATIVAS_DEVOLUCAO
		ON produtos.id_chave = ATIVAS_DEVOLUCAO.id_chave
) AS PRODUTOS_ATIVOS_DEVOLVIDOS
WHERE PRODUTOS_ATIVOS_DEVOLVIDOS.id_produto IS NOT NULL
"""

In [10]:
query_produtos_devolvidos = \
rf"""
SELECT
	SUM(produtos.quantidade) AS qtde_produtos_devolvidos
FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_PRODUTO AS produtos -- Carregando a tabela produtos que é o alvo da query
INNER JOIN (
	SELECT
		MIN(chave.id_chave) AS id_chave,
		chave.chave
	FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
	INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
		ON nfe.id_chave = chave.id_chave
	INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
		ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
	INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
		ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
	WHERE
		nfe.nota_cancelada = 0 AND
		nfe.id_finalidade = 4 AND
		cpf_cnpj.cnpj = {cnpj} AND
		chave.chave NOT IN (
			SELECT
				DISTINCT chave.chave
			FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
			INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
				ON nfe.id_chave = chave.id_chave
			INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
				ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
			INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
				ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
			WHERE
				nfe.nota_cancelada = 1 AND
				cpf_cnpj.cnpj = {cnpj}
			)
		GROUP BY chave.chave
) AS ATIVAS_DEVOLUCAO
	ON produtos.id_chave = ATIVAS_DEVOLUCAO.id_chave
WHERE produtos.quantidade IS NOT NULL
"""

### Resultados

In [11]:
with engine.begin() as conn:
    df_notas_canceladas = pd.read_sql_query(text(query_notas_canceladas), conn)

    df_notas_ativas = pd.read_sql_query(text(query_notas_ativas), conn)
    df_itens_ativos = pd.read_sql_query(text(query_itens_ativos), conn)
    df_produtos_ativos = pd.read_sql_query(text(query_produtos_ativos), conn)

    df_notas_devolvidas = pd.read_sql_query(text(query_notas_devolvidas), conn)
    df_itens_devolvidos = pd.read_sql_query(text(query_itens_devolvidos), conn)
    df_produtos_devolvidos = pd.read_sql_query(text(query_produtos_devolvidos), conn)

In [12]:
df_final = pd.DataFrame({
    "Qtde notas canceladas": df_notas_canceladas["qtde_notas_canceladas"].values,
    "Qtde notas ativas": df_notas_ativas["qtde_notas_ativas"].values,
    "Qtde itens ativos": df_itens_ativos["qtde_itens_ativos"].values,
    "Qtde produtos ativos": df_produtos_ativos["qtde_produtos_ativos"].values,
    "Qtde notas devolvidas": df_notas_devolvidas["qtde_notas_devolvidas"].values,
    "Qtde itens devolvidos": df_itens_devolvidos["qtde_itens_devolvidos"].values,
    "Qtde produtos devolvidos": df_produtos_devolvidos["qtde_produtos_devolvidos"].values
})
df_final

,Qtde notas canceladas,Qtde notas ativas,Qtde itens ativos,Qtde produtos ativos,Qtde notas devolvidas,Qtde itens devolvidos,Qtde produtos devolvidos
0,5,48,95,11765.0,3,8,2470.0
